# Optimization with JuMP

Optimization models play a central role in operations research. 

In this session, we will explore the basic functionalities of JuMP, a modeling language to define and solve optimization models in Julia.

<a href="https://github.com/JuliaOpt/JuMP.jl"><img src="figures/jump.svg" alt="Julia" style="width: 500px;"/></a>

Suppose Iain wants to carry items to the pawn shop to get some extra cash. He has $N$ items, each with a weight $w_i$ and a price $p_i$. Iain hasn't been to the gym lately, so he can only carry $C$ kilos. How does he choose what to bring with him?

We can model this as an integer optimization problem:

\begin{align*}
\max& \sum_{i=1}^N p_i x_i \\
\text{s.t.}& \sum_{i=1}^N w_i x_i \leq C \\
& x_i \in \{0,1\} \quad \forall i = 1,\ldots,N
\end{align*}



## A toy example

Consider the following (small) knapsack problem:

\begin{align*}
    \max\:& x + y \\
    \text{s.t.}\:& x + 2y \leq 1.5 \\
    & x, y \in \{0,1\}
\end{align*}

How would you solve this? 

## Exhaustive enumeration

The simple way is just to consider each possible value for $x$ and $y$ and compare the cost.

![alt text](figures/tree_1.png)

In the general case, this would lead to $2^N$ possible collections of items. After Iain has weighed all of them (and verified that he can lift them at once), he just chooses the best set.

## Branch-and-bound: smart enumeration
Let's visualize this approach as a search tree:

![alt text](figures/tree_2.png)

Each node is a subproblem with 
- some variables fixed to $0$
- some variables fixed to $1$
- the remaining variables are unknown 

At each node, we have techniques to 
- find a lower bound on the objective value (i.e., find a feasible solution)
- find an upper bound on the objective value (e.g., solve the relaxation)

This leads us to a powerful tool in solving these enumeration problems: 
>If I can show you that the optimal cost for subproblem ``q`` is _less_ than the optimal cost for the original problem, the same is true for any descendent of ``q``. 


That is, we can _prune_ the tree and safely discard some nodes, kind of like this:

![alt text](figures/tree_3.png)

# Implementation in JuMP

Let's solve our simple knapsack problem in JuMP.

[JuMP.jl](https://github.com/JuliaOpt/JuMP.jl) is a high-level modeling language for mathematical optimization embedded in Julia. It currently supports 
- a number of open-source and commercial solvers (Artelys Knitro, BARON, Bonmin, Cbc, Clp, Couenne, CPLEX, ECOS, FICO Xpress, GLPK, Gurobi, Ipopt, MOSEK, NLopt, SCS) 
- for a variety of problem classes, including linear programming, (mixed) integer programming, second-order conic programming, semidefinite programming, and nonlinear programming.

<a href="http://www.juliaopt.org"><img src="figures/juliaopt.png" alt="JuliaOpt" style="width: 1000px;"/></a>

Take a simple knapsack problem 

\begin{align*}
    \max\:& x + y \\
    \text{s.t.}\:& x + 2y \leq 1.5 \\
    & x, y \in \{0,1\}
\end{align*}

you can easily define it in JuMP:

In [ ]:
using JuMP, Gurobi
m = Model(solver=GurobiSolver())

@variable(m, x, Bin) # or Int
@variable(m, y, Bin)

@constraint(m, x + 2y ≤ 1.5)
@objective(m, Max, x + y)

print(m)

and solve it

In [ ]:
solve(m)

This is kind of dull since Gurobi solves this before we ever get to the branch-and-bound tree! Let's cook up a problem that's a little more interesting. What about more items, and more knapsacks! If $N=100$, naïve enumeration would create $2^{100}$ nodes, which would take quite some time. How does the solver actually tackle it?

In [ ]:
using LinearAlgebra
N = 100

m = Model(solver=GurobiSolver())
@variable(m, x[1:N], Bin)
for _ in 1:10
    @constraint(m, dot(rand(N), x) ≤ N / 50)
end

@objective(m, Max, dot(rand(N), x))

solve(m)

## Sudoku

![Sudoku](http://upload.wikimedia.org/wikipedia/commons/f/ff/Sudoku-by-L2G-20050714.svg)

**Sudoku** is a number puzzle played on a 9x9 grid. The challenge is to place a digit between 1 and 9 inclusive in each empty cell, such that the completed grid obeys the following rules:

* Each row contains the numbers 1 to 9 once and only once.
* Each column contains the numbers 1 to 9 once and only once.
* Each 3x3 subgrid contains the numbers 1 to 9 once and only once.

>**\[Exercise\]**: Sudoku

> Model the Sudoku problem as an optimization problem.

> Write a Julia function to solve it.

** Variables **

We define binary variables 
$$ x_{i,j,k} \in \{0, 1\}, $$
for each row $i$, column $j$ and integer $k \in \{ 1,2, \dots,9\}$.

$x_{i,j,k} = 1$ iff the number $k$ appears in cell $(i,j)$

** Constraints **

** Objective **

Let us solve this problem in Julia now!

In [ ]:
init_vals = [
5 3 0 0 7 0 0 0 0
6 0 0 1 9 5 0 0 0
0 9 8 0 0 0 0 6 0
8 0 0 0 6 0 0 0 3
4 0 0 8 0 3 0 0 1
7 0 0 0 2 0 0 0 6
0 6 0 0 0 0 2 8 0
0 0 0 4 1 9 0 0 5
0 0 0 0 8 0 0 7 9
]

using PyPlot
# Displays a sudoku matrix. If an initial state is provided, 
# then the new numbers are displayed in red. 
function display_sudoku(sudoku, initial_state=zeros(9,9))
    fig, ax = subplots()
    ax[:axis]("off")
    # Make sudoku basic 9x9 grid.
    for i in 0:9
        plot([i,i], [0, 9], color = "black")
        plot([0, 9], [i,i], color = "black")
    end
    # Make thicker lines to separate the 3x3 subgrids.
    for i in 0:3:9
        plot([i,i], [0,9], color = "black", linewidth = 3)
        plot([0,9], [i,i], color = "black", linewidth = 3)
    end
    # Display the values at the right square.
    for i in 0:8
        for j in 0:8 
            value = Int(sudoku[9-j,i+1])
            old_value = initial_state[9-j,i+1]
            # If an initial state is not provided (set to zero) or the value of the 
            # square did not change, the color is set to black. Otherwise, it becomes red.
            col = (sum(initial_state) == 0 || old_value == value) ? "black" : "red"
            if value > 0 # Omit zero values  
                text(i + 0.35, j + 0.33, value, size = 16, color = col)
            end
        end
    end
end
display_sudoku(init_vals)

In [ ]:
sudoku = Model(solver=GurobiSolver())

#write your code here

print(sudoku)

In [ ]:
solve(sudoku)

In [ ]:
sol = sum(i * getvalue(x)[:,:,i] for i in 1:9)

In [ ]:
display_sudoku(sol, init_vals)